<a href="https://colab.research.google.com/github/EoghanPower/pr/blob/main/Create_a_Tweets_DataFrame2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [36]:

#Creating a Data Frame of Tweets on S&P 500 Companies

In [37]:
# import libraries
import tweepy, json 
from textblob import TextBlob 
from wordcloud import wordcloud
import pandas as pd
import numpy as np 
import re
import matplotlib.pyplot as plt 
plt.style.use('ggplot')
import webbrowser
import time 
import requests
import requests_oauthlib
import os

In [38]:
#authentication store 0auth credentials in variables
consumer_key = 'eHrpqTZVq5bfGdjC8juN7fSCs'
consumer_secret = 'itfojeERaMFXtwQFI1emQNT3vwCHs414OJdOXlpN2RaBAw3MIz'
AccessToken = '264845011-WhC9gKJlC3TayNsXg4ViBIuwaK0D5Sof3lxPTZeN'
AccessTokenSecret = 'dE5difJJmYP5UYllqeQwTsovcoXyvco9Ww2QMxiR365D3'

In [39]:
#Pass 0auth details to tweepy's 0auth handlet 
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(AccessToken, AccessTokenSecret)

In [40]:
##SetAPIobject
api = tweepy.API(auth, wait_on_rate_limit= True)

In [41]:
#StreamListener
class MyStreamListener(tweepy.StreamListener):
    def __init__(self, api=None):
        super(MyStreamListener, self).__init__()
        self.num_tweets = 0
        self.file = open("tweets.txt", "w")

    def on_status(self, status):
        tweet = status._json
        self.file.write( json.dumps(tweet) + '\n' )
        self.num_tweets += 1
        if self.num_tweets < 10001:
            return True
        else:
            return False
        self.file.close()

    def on_error(self, status):
        print(status)

In [43]:
#initialize stream listener
l = MyStreamListener()
#Create stream object with authentication
stream = tweepy.Stream(auth, l)
#Filter Twitter Streams to capture data by the keywords:
stream.filter(languages = ['en'],track=['Apple', 'Google', 'Microsoft','Amazon',  'Facebook', 'JPMorgan',   'Johnson & Johnson', '  Exxon', 'Bank of America', 'Wal-Mart, Wells Fargo', 'Berkshire Hathaway', 'AT&T ', 'Tesla', 'Gamestop' ])

In [44]:
#import package
import json

#String of path to file: tweets_data_path
tweets_data_path = 'tweets.txt'

#initialize empty list to store tweets: tweets_data
tweets_data = []

In [45]:
#open connection to file
tweets_file = open(tweets_data_path, 'r')

In [46]:
# Read in tweets and store in list: tweets_data
for line in tweets_file:
  tweet = json.loads(line)
  tweets_data.append(tweet)

In [47]:
#close connection to file
tweets_file.close()


In [48]:
print(tweets_data[0].keys())

dict_keys(['created_at', 'id', 'id_str', 'text', 'source', 'truncated', 'in_reply_to_status_id', 'in_reply_to_status_id_str', 'in_reply_to_user_id', 'in_reply_to_user_id_str', 'in_reply_to_screen_name', 'user', 'geo', 'coordinates', 'place', 'contributors', 'retweeted_status', 'is_quote_status', 'quote_count', 'reply_count', 'retweet_count', 'favorite_count', 'entities', 'favorited', 'retweeted', 'filter_level', 'lang', 'timestamp_ms'])


In [50]:
# Build DataFrame of tweet texts and languages
df = pd.DataFrame(tweets_data, columns= ['text'])

In [51]:
print(df.head(10))

                                                text
0  RT @bopinion: .@elonmusk is now, on paper at l...
1  RT @NationAfrica: Murang’a boda boda rider hel...
2  @_centralbts @BTS_twt Google https://t.co/UfiY...
3  @MoreAbtNothng Lol them dudes on Facebook a tr...
4  RT @LucyENichol: @samatlounge I self-released ...
5  RT @LeannaOrganics: "Imagine a man without lun...
6  RT @OddlyB: I just found out my niece &amp; he...
7  RT @dcboxley: DH2i Just Announced -- https://t...
8  RT @TheRickWilson: This is your periodic remin...
9  The City of Greater Sudbury has opened one of ...


In [52]:
#clean the tweets
def cleanTwt(twt): 
  twt = re.sub('#apple','Apple', twt)
  twt = re.sub('#google','Google', twt) 
  twt = re.sub('#microsoft','Microsoft', twt) 
  twt = re.sub('#amazon','Amazon', twt) 
  twt = re.sub('#facebook','Facebook', twt) 
  twt = re.sub('#JPMorgan','JPMorgan', twt) 
  twt = re.sub('#Johnson&Johnson','Johnson & Johnson', twt) 
  twt = re.sub('#Exxon','Exxon', twt) 
  twt = re.sub('#BankofAmerica','Bank of America', twt) 
  twt = re.sub('#Wal-Mart','Wal-Mart', twt) 
  twt = re.sub('#Wells Fargo','Wells Fargo', twt) 
  twt = re.sub('#BerkshireHathaway','Berkshire Hathaway', twt) 
  twt = re.sub('#Gamestop','Gamestop', twt) 
  twt = re.sub('#Tesla ','Tesla', twt)#removes the # 
  twt =re.sub('#[A-Za-z0-9]+', '',twt) #removes any strings with a # 
  twt = re.sub('\\n', '', twt) #removes the '\n' string
  twt = re.sub('https?:\/\/\S+', '', twt) #removes any hyperlinks
  twt = re.sub('RT', '', twt) #cleans retweets
  twt = re.sub('@[A-Za-z0-9]+', ',', twt)#Removes tagged accounts
  return twt

In [53]:
#clean them using the above function

In [54]:
df['Cleaned_Tweets'] = df['text'].apply(cleanTwt)

In [55]:
#show the set 
df.head(10)

,text,Cleaned_Tweets
0,"RT @bopinion: .@elonmusk is now, on paper at l...",",: ., is now, on paper at least, the richest ..."
1,RT @NationAfrica: Murang’a boda boda rider hel...,",: Murang’a boda boda rider held over death o..."
2,@_centralbts @BTS_twt Google https://t.co/UfiY...,"@_centralbts ,_twt Google"
3,@MoreAbtNothng Lol them dudes on Facebook a tr...,", Lol them dudes on Facebook a trip 😅"
4,RT @LucyENichol: @samatlounge I self-released ...,",: , I self-released my first novel, The Twen..."
5,"RT @LeannaOrganics: ""Imagine a man without lun...",",: ""Imagine a man without lungs. Imagine eart..."
6,RT @OddlyB: I just found out my niece &amp; he...,",: I just found out my niece &amp; her husban..."
7,RT @dcboxley: DH2i Just Announced -- https://t...,",: DH2i Just Announced -- DH2i and Pythian t..."
8,RT @TheRickWilson: This is your periodic remin...,",: This is your periodic reminder that , coul..."
9,The City of Greater Sudbury has opened one of ...,The City of Greater Sudbury has opened one of ...


In [56]:
#create a function to get subjectivity
def getSubjectivity(twt):
  return TextBlob(twt).sentiment.subjectivity
#create a function to get polarity
def getPolarity(twt):
  return TextBlob(twt).sentiment.polarity


In [57]:
#create two new columns called subjectivity and polarity
df['Subjectivity'] = df['Cleaned_Tweets'].apply(getSubjectivity)
df['Polarity'] = df['Cleaned_Tweets'].apply(getPolarity)

In [58]:
#show data
df.head(20)

,text,Cleaned_Tweets,Subjectivity,Polarity
0,"RT @bopinion: .@elonmusk is now, on paper at l...",",: ., is now, on paper at least, the richest ...",0.450000,0.100000
1,RT @NationAfrica: Murang’a boda boda rider hel...,",: Murang’a boda boda rider held over death o...",0.000000,0.000000
2,@_centralbts @BTS_twt Google https://t.co/UfiY...,"@_centralbts ,_twt Google",0.000000,0.000000
3,@MoreAbtNothng Lol them dudes on Facebook a tr...,", Lol them dudes on Facebook a trip 😅",0.700000,0.800000
4,RT @LucyENichol: @samatlounge I self-released ...,",: , I self-released my first novel, The Twen...",0.200000,0.041667
5,"RT @LeannaOrganics: ""Imagine a man without lun...",",: ""Imagine a man without lungs. Imagine eart...",0.000000,0.000000
6,RT @OddlyB: I just found out my niece &amp; he...,",: I just found out my niece &amp; her husban...",0.000000,0.000000
7,RT @dcboxley: DH2i Just Announced -- https://t...,",: DH2i Just Announced -- DH2i and Pythian t...",0.300000,0.178788
8,RT @TheRickWilson: This is your periodic remin...,",: This is your periodic reminder that , coul...",0.535714,0.285714
9,The City of Greater Sudbury has opened one of ...,The City of Greater Sudbury has opened one of ...,0.500000,0.500000


In [59]:
#Create a function to get the sentiment text
def getSentiment(score):
  if score <0:
    return 'Negative'
  elif score == 0:
      return 'Neutral'
  else:
      return 'Positive'

In [60]:
#Create column to store the text sentiment
df['Sentiment'] = df['Polarity'].apply(getSentiment)
#Show the data
df.head(7)

,text,Cleaned_Tweets,Subjectivity,Polarity,Sentiment
0,"RT @bopinion: .@elonmusk is now, on paper at l...",",: ., is now, on paper at least, the richest ...",0.45,0.100000,Positive
1,RT @NationAfrica: Murang’a boda boda rider hel...,",: Murang’a boda boda rider held over death o...",0.00,0.000000,Neutral
2,@_centralbts @BTS_twt Google https://t.co/UfiY...,"@_centralbts ,_twt Google",0.00,0.000000,Neutral
3,@MoreAbtNothng Lol them dudes on Facebook a tr...,", Lol them dudes on Facebook a trip 😅",0.70,0.800000,Positive
4,RT @LucyENichol: @samatlounge I self-released ...,",: , I self-released my first novel, The Twen...",0.20,0.041667,Positive
5,"RT @LeannaOrganics: ""Imagine a man without lun...",",: ""Imagine a man without lungs. Imagine eart...",0.00,0.000000,Neutral
6,RT @OddlyB: I just found out my niece &amp; he...,",: I just found out my niece &amp; her husban...",0.00,0.000000,Neutral


In [ ]:
df.drop(columns = 'text')

In [63]:
#drop columns to make it easier to read
df.head()

,Cleaned_Tweets,Subjectivity,Polarity,Sentiment
0,",: ., is now, on paper at least, the richest ...",0.45,0.100000,Positive
1,",: Murang’a boda boda rider held over death o...",0.00,0.000000,Neutral
2,"@_centralbts ,_twt Google",0.00,0.000000,Neutral
3,", Lol them dudes on Facebook a trip 😅",0.70,0.800000,Positive
4,",: , I self-released my first novel, The Twen...",0.20,0.041667,Positive
